# 3D Mesh Classification — week 03

I worked on a 3D mesh classification task using Python to categorize STL models as either Flat or Free-Form. For this, I used libraries such as trimesh, numpy, pandas, and sklearn.cluster (specifically KMeans). I tested five different geometric methods to analyze the mesh shapes and later selected the two most effective ones—Height Ratio (HR) and Aspect Ratio (AR)—for the final filtering process.

## 1. Height Ratio (HR) Method

In this method, I calculated the height (Z), width (X), and depth (Y) from each mesh’s bounding box. The formula used was HR = H / max(W, D). If the HR value was less than 0.2, the model was classified as Flat; otherwise, it was categorized as Free-Form. This approach performed very well, being fast to compute and clearly distinguishing flat models such as panels. Its accuracy level was high.

## 2. Normal Vector Variation (NV) Method

This method involved calculating all face normals from the mesh and finding the average normal direction. Then, I measured the average angle difference between each normal and the mean. When NV was below 10°, the mesh was considered Flat, while values equal to or above 10° indicated Free-Form. Although the method worked, some meshes produced incorrect results due to surface noise, giving it a medium accuracy level.

## 3. Normalized Z Variation (ZVar) Method

In this approach, I took all vertex Z-values, computed their standard deviation, and divided it by the total height of the model. The formula was ZVar = std(Z) / H. Models with ZVar less than 0.05 were classified as Flat, and those with ZVar equal to or above 0.05 were Free-Form. This method was effective for simple flat surfaces but struggled with rotated or tilted models, resulting in medium accuracy.

## 4. Planarity Dominance (PD) Method

This method relied on K-Means clustering applied to face normals to identify the largest planar area. The ratio of the largest plane area to the total surface area was computed using PD = max_plane_area / total_area. If PD was greater than 0.8, the mesh was labeled Flat; otherwise, it was Free-Form. While this method worked well in many cases, it was slower and sometimes inconsistent depending on the clustering outcome. Its accuracy level was variable.

## 5. Aspect Ratio (AR) Method

The aspect ratio was computed from the bounding box dimensions of the mesh as AR = min(size) / max(size). Models with AR values below 0.15 were classified as Flat, and those equal to or above 0.15 were Free-Form. This method was extremely fast, stable, and worked correctly even for rotated meshes. It provided consistently high accuracy.

## 6. Results and Findings

•	Height Ratio (HR) and Aspect Ratio (AR) gave the best accuracy.
•	Both are easy to compute and don’t depend on mesh quality or rotation.
•	NV, ZVar, and PD were useful but less stable.
•	I finally used only HR and AR for final filtering.
•	Accuracy level was very high with these two methods.

Metric	Accuracy	Comment
Height Ratio	High	Clear for flat vs thick models
Aspect Ratio	High	Works well for rotated meshes
Normal Variation	Medium	Noise sensitive
Z Variation	Medium	Works for simple planes
Planarity Dominance	Variable	Depends on clustering



In [3]:
import os
import trimesh
import numpy as np
import pandas as pd

# === Step 1: Get all STL files in a folder ===
def get_stl_files(folder_path):
    stl_files = [os.path.join(folder_path, f) 
                 for f in os.listdir(folder_path) 
                 if f.lower().endswith('.stl')]
    return stl_files

# === Step 2: Compute Aspect Ratio (Shape Slenderness) ===
def compute_aspect_ratio(mesh):
    bounds = mesh.bounds  # [[minx, miny, minz], [maxx, maxy, maxz]]
    size = bounds[1] - bounds[0]  # [width, depth, height]

    width, depth, height = size[0], size[1], size[2]
    max_horizontal = max(width, depth)

    if max_horizontal == 0:
        return 0

    aspect_ratio = height / max_horizontal
    return aspect_ratio

# === Step 3: Process STL files ===
def process_stl_files(stl_files, output_csv):
    results = []
    for file_path in stl_files:
        try:
            mesh = trimesh.load_mesh(file_path)
            if not isinstance(mesh, trimesh.Trimesh):
                continue

            aspect_ratio = compute_aspect_ratio(mesh)
            results.append({
                "FileName": os.path.basename(file_path),
                "AspectRatio": aspect_ratio
            })
        except Exception as e:
            print(f"Error processing {file_path}: {e}")

    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"✅ CSV saved to: {output_csv}")

# === Step 4: Run the pipeline ===
if __name__ == "__main__":
    stl_folder = r"D:\research\week03\Presha"  # <-- Change to your folder path
    output_csv = "aspect_ratio.csv"

    stl_files = get_stl_files(stl_folder)
    process_stl_files(stl_files, output_csv)


✅ CSV saved to: aspect_ratio.csv


In [5]:
import os
import trimesh
import pandas as pd

# --------------------- STEP 1: Compute Height Ratio ---------------------
def compute_height_ratio(mesh):
    bounds = mesh.bounds  # [[x_min, y_min, z_min], [x_max, y_max, z_max]]
    x_min, y_min, z_min = bounds[0]
    x_max, y_max, z_max = bounds[1]

    H = z_max - z_min  # height
    W = x_max - x_min  # width
    D = y_max - y_min  # depth

    max_width_depth = max(W, D)
    if max_width_depth == 0:
        return 0  # avoid division by zero

    HR = H / max_width_depth
    return HR

# --------------------- STEP 2: Classification ---------------------
def classify_height_ratio(hr, threshold=0.2):
    return "Flat" if hr < threshold else "Freeform"

# --------------------- STEP 3: Process Folder ---------------------
def process_stl_folder(folder_path, output_csv="height_ratio_results.csv"):
    results = []

    # Walk through folder and find .stl files
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(".stl"):
                file_path = os.path.join(root, file)
                try:
                    mesh = trimesh.load(file_path, force='mesh')
                    hr = compute_height_ratio(mesh)
                    classification = classify_height_ratio(hr)
                    results.append({
                        "FileName": file,
                        "HeightRatio": round(hr, 4),
                        "Classification": classification
                    })
                    print(f"[✅] {file} → HR={hr:.4f} → {classification}")
                except Exception as e:
                    print(f"[❌] Error processing {file}: {e}")

    # Save all results to CSV
    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"\n📄 Results saved to: {output_csv}")

# --------------------- MAIN ---------------------
if __name__ == "__main__":
    # 🔸 Set your STL folder path here
    folder_path = r"D:\research\week03\Presha"   # ⬅️ Change this!

    process_stl_folder(folder_path, output_csv="height_ratio_results.csv")


[✅] 103742.stl → HR=0.0185 → Flat
[✅] 103815.stl → HR=0.0777 → Flat
[✅] 103817.stl → HR=0.6421 → Freeform
[✅] 103821.stl → HR=1.9000 → Freeform
[✅] 103824.stl → HR=3.2510 → Freeform
[✅] 103825.stl → HR=3.2510 → Freeform
[✅] 103826.stl → HR=1.4549 → Freeform
[✅] 104187.stl → HR=0.1534 → Flat
[✅] 104188.stl → HR=0.1112 → Flat
[✅] 40893.stl → HR=0.2082 → Freeform
[✅] 40900.stl → HR=0.6694 → Freeform
[✅] 40906.stl → HR=0.2316 → Freeform
[✅] 40915.stl → HR=0.2292 → Freeform
[✅] 40920.stl → HR=0.1085 → Flat
[✅] 40921.stl → HR=0.0632 → Flat
[✅] 40923.stl → HR=1.3113 → Freeform
[✅] 40926.stl → HR=0.9985 → Freeform
[✅] 40982.stl → HR=0.7100 → Freeform
[✅] 40984.stl → HR=1.7437 → Freeform
[✅] 40985.stl → HR=1.7515 → Freeform
[✅] 40987.stl → HR=2.3946 → Freeform
[✅] 40991.stl → HR=0.3179 → Freeform
[✅] 40992.stl → HR=0.7375 → Freeform
[✅] 40993.stl → HR=0.7705 → Freeform
[✅] 40994.stl → HR=0.4354 → Freeform
[✅] 41077.stl → HR=0.7155 → Freeform
[✅] 41079.stl → HR=1.7125 → Freeform
[✅] 41080.stl → 